In [1]:
import time
import os
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import json
from activity_detector import ActivityDetector, LateralActivityHost, LateralActivityTarget, \
    LeadVehicle, ActivityDetectorParameters, LineData
from find_index_from_video import approx_index
from ngram import NGram
import seaborn as sns
from tqdm import tqdm
from typing import Iterable, List, NamedTuple, Tuple
%matplotlib inline
#%load_ext autoreload
#%autoreload 2

In [2]:
# Load the data
i_file = 0
datafiles = glob(os.path.join("data", "1_hdf5", '*.hdf5'))
AD = ActivityDetector(datafiles[i_file])

In [ ]:
plt.plot(AD.targets[5191]['age'])

In [ ]:
AD.set_lon_activities_host()

In [ ]:
AD.set_lat_activities_host()

In [ ]:
events = AD.lat_activities_host()

In [ ]:
i = 1
window = 5
print(events[i])
print(events[i+1])
plt.plot(AD.data.loc[events[i][0]-window:events[i+1][0]+window, AD.parms.y_left_line])
plt.plot(AD.data.loc[events[i][0]-window:events[i+1][0]+window, AD.parms.y_right_line])
ylim = plt.ylim()
plt.plot([events[i][0], events[i][0]], ylim, 'k')
plt.plot([events[i+1][0], events[i+1][0]], ylim, 'k')
plt.ylim(ylim)

The following codes checks the recall of the lane changed detection. It shows the activity at the index at which a lane change occurs. These indices are obtained after looking at the video. 

It shows that all left lane changes are detected. Two right lane changes are missed. 

In [ ]:
# Jeroen's code does not detect the lane change at 1963.60.
i_left_lane_change = [203.35, 217.29, 1738.83, 1813.3, 1900.71, 1963.60,
                      2000.3, 2171.98, 2310.65, 2318.23, 2546.01, 2650.02]
print([AD.get("host_lateral_activity", i) for i in i_left_lane_change])
# Jeroen's code does not detect the lane change at 2193.44 and 2236.72.
# These lane changes are also difficult to see by eye, so these indices
# are approximate.
i_right_lane_change = [251.46, 304.95, 1846.39, 1858.69, 1927.05, 2040.03,
                       2193.44, 2236.72, 2487.13, 2568.65, 2668.15, 2675.00]
print([AD.get("host_lateral_activity", i) for i in i_right_lane_change])

The following code shows the recall of the cut-in detection at the highway. There are six cut-ins identified from the videos. The lateral activities for these six target vehicles are determined. 
All cut-ins are detected. For some strange reason, the lane changes are detected 3 seconds before the timing that is obtained from the video. That might be a syncing error?

In [ ]:
# Cut-in targets:
# 3850: Motorbike from right (29:47, index=1802)
# 4171: Dark passenger car from right (quite far) (32:50, index=1985)
# 5028: Silver passenger car from right (39:18, index=2373)
# 5191: Black van from left (40:53, index=2468)
# 5645: Black passenger car from left (42:26, index=2562)
# 6008: Gray VW from left (44:26, index=2681)
for i_target, index in zip([3850, 4171, 5028, 5191, 5645, 6008],
                           [1802., 1985., 2373., 2468., 2562., 2681.]):
    AD.set_target_activities(i_target)
    print("Activity of target {:d} at index {:.2f}: {:s}".
          format(i_target, index-3, AD.targets[i_target].at[index-3, "lateral_activity"]))

Compute all events for the targets. This takes quite some time, around 13 minutes on the zBook laptop.

In [ ]:
# This takes round 13 minutes!
AD.set_target_activities()

Set the tags for the longitudinal and lateral state of each target vehicle at each time instant. Furthermore, tags are added that specify whether a target vehicle is a lead vehicle at a certain time instant. The states of the targets (i.e., `set_states_targets()`) can only be done after the lateral events (through `set_target_activities()`), because the lateral position of the targets with respect to the lane lines is needed. Similarly, the determination of the lead vehicle (`set_lead_vehicle()`) can only be done after the other states (`set_states_targets()`), because it makes use of the result of the previous step.

`set_states_targets()` is really fast, `set_lead_vehicle()` takes around 3 minutes on the zBook laptop.

In [ ]:
AD.set_states_targets()
AD.set_lead_vehicle()

## Create or load n-grams for target vehicles

In [3]:
filename = os.path.join("data", "4_ngrams", "{:s}_targets.hdf5".
                        format(os.path.splitext(os.path.basename(datafiles[i_file]))[0]))
fieldnames = ["longitudinal_activity", "lateral_activity", "longitudinal_state", 
              "lateral_state", "lead_vehicle", "id"]
metadata = (("tstart", float), ("tend", float), ("target_id", int))
target_ngrams = NGram(fieldnames, metadata)

In [4]:
if not target_ngrams.from_hdf(filename, "targets"):
    for target in AD.targets:
        target_ngrams.ngram_from_data(target, 
                                      tstart=target.index[0],
                                      tend=target.index[-1],
                                      target_id=int(target["id"].values[0]))
    target_ngrams.sort_ngrams("tstart")
    target_ngrams.to_hdf(filename, "targets", mode="w")

## Create or load n-gram for ego vehicle

In [5]:
fieldnames = ["host_longitudinal_activity", "host_lateral_activity", "is_highway"]
metadata = (("tstart", float), ("tend", float))
ego_ngram = NGram(fieldnames, metadata)

In [6]:
if not ego_ngram.from_hdf(filename, "ego"):
    ego_ngram.ngram_from_data(AD.data, tstart=AD.data.index[0], tend=AD.data.index[-1])
    ego_ngram.to_hdf(filename, "ego")

## Extract cut-in scenario

For target vehicle, we need:
1. Lateral activity `li` or `ri`.
2. Lateral activity `fl` and lead vehicle `y`.

For the ego vehicle, we need it to go straight during step 2 of the target vehicle.

In [8]:
def check_row(row, dict_tags: dict) -> bool:
    """ Check if a row of a dataframe contains the provided tags. 
    
    Each item of the dictionary needs to contain a list. 
    
    :param row: The row that is obtained through pd.DataFrame.itertuples().
    :param dict_tags: The dictionary of tags.
    """
    for key, tags in dict_tags.items():
        if getattr(row, key) not in tags:
            return False
    return True

In [9]:
target_tags = [dict(lateral_activity=[LateralActivityTarget.LEFT_CUT_IN.value, 
                                      LateralActivityTarget.RIGHT_CUT_IN.value]),
               dict(lateral_activity=[LateralActivityTarget.LANE_FOLLOWING.value], 
                    lead_vehicle=[LeadVehicle.LEAD.value])]
ego_tags = [dict(host_lateral_activity=[LateralActivityHost.LANE_FOLLOWING.value],
                 is_highway=[True]),
            dict(host_lateral_activity=[LateralActivityHost.LANE_FOLLOWING.value],
                 is_highway=[True])]

In [10]:
class _NGramSearch(NamedTuple):
    is_found: bool
    index: int = 0
    time: float = 0.0
        
class _StartEnd(NamedTuple):
    is_found: bool
    t_start: float = None
    t_end: float = None

def determine_start(ngram: pd.DataFrame, tags: Iterable[dict], 
                    t_start: float = None, tend: float = None) -> _NGramSearch:
    # Check if the n-gram has data within [tstart, tend].
    if t_start is not None and tend is not None:
        if t_start > ngram.index[-1] or tend < ngram.index[0]:
            return _NGramSearch(False)
    
    i_start = 0
    if t_start is not None and t_start > ngram.index[0]:
        i_start = ngram.index.get_loc(t_start, method='pad')
        ngram = ngram.iloc[i_start:]
    for i, row in enumerate(ngram.itertuples(), start=i_start):
        if check_row(row, tags):
            break
    if t_start is not None and t_start > row.Index:
        return _NGramSearch(True, i, t_start)
    return _NGramSearch(True, i, row.Index)
        
def determine_end(ngram, tags, istart, tend=None):
    row = None
    for i, row in enumerate(ngram.iloc[istart+1:].itertuples(), start=istart+1):
        if not check_row(row, tags):
            break
        if tend is not None and row.Index > tend:
            break
    if row is None:  # This might happen if istart+1 == len(ngram).
        return _NGramSearch(False)
    if tend is not None and tend < row.Index:
        return _NGramSearch(True, i, tend)
    return _NGramSearch(True, i, row.Index)

def determine_start_end(ngram, tags, previous_search=None):
    if previous_search is None:
        previous_search = _StartEnd(False)
    start = determine_start(ngram, tags, previous_search.t_start, previous_search.t_end)
    if not start.is_found:
        return _StartEnd(False)
    end = determine_end(ngram, tags, start.index, previous_search.t_end)
    if not end.is_found:
        return _StartEnd(False)
    return _StartEnd(True, start.time, end.time)

def find_part_of_sequence(ngrams: Iterable[pd.DataFrame], tags: Iterable[dict],
                          t_start: float = None, force_start: bool = False):
    # Check for the first tag of first n-gram.
    searches = np.zeros(len(ngrams), dtype=_StartEnd)
    searches[0] = _StartEnd(True, t_start, ngrams[0].index[-1])
    level = 0
    while True:
        # Four possible results:
        # 1. Tag found and not at highest level.
        # 2. Tag found and at highest level, so return True.
        # 3. Tag not found and not at lowest level, so go one level up and start search from the 
        #    previous end.
        # 4. Tag not found and at lowest level, so return False.
        search = determine_start_end(ngrams[level], tags[level], previous_search=searches[level])
        if search.is_found:  # Possibility 1 or 2.
            #print("0, {:d}: tstart={:.2f}, tend={:.2f}".format(level, search.t_start, search.t_end))
            if force_start and search.t_start > t_start:
                return _StartEnd(False)
            level += 1
            if level < len(ngrams):  # Possibility 1.
                searches[level] = search
            else:  # Possibility 2.
                return _StartEnd(True, search.t_start, search.t_end)
        else:  # Possibility 3 or 4.
            level -= 1
            if level >= 0:  # Possibility 3.
                # We need to go one level up and start searching from the previous end
                # to see if we can find a new match. However, it might be possible that
                # the new window has length 0. In that case, we need to go one level up.
                # This might continue until we reach the lowest level. In that case, we
                # will not find a match, so we can return a False
                while searches[level+1].t_end >= searches[level].t_end:
                    level -= 1
                    if level < 0:
                        return _StartEnd(False)
                searches[level] = _StartEnd(False, searches[level+1].t_end, searches[level].t_end)
            else:  # Possibility 4.
                return _StartEnd(False)

def find_sequence(ngrams: Iterable[pd.DataFrame], tags: Iterable[Iterable[dict]], 
                  t_start: float = None) -> _StartEnd:
    # Check for the first tag of first n-gram.
    search = find_part_of_sequence(ngrams, [tag[0] for tag in tags], t_start)
    if not search.is_found:
        return _StartEnd(False)
    t_start = search.t_start

    # Go through remaining steps.
    for j in range(1, len(tags[0])):
        search = find_part_of_sequence(ngrams, [tag[j] for tag in tags], search.t_end,
                                       force_start=True)
        if not search.is_found:
            return _StartEnd(False)
    return _StartEnd(True, t_start, search.t_end)

t = target_ngrams.ngrams[554]
e = ego_ngram.ngram
print(find_sequence((t, e), (target_tags, ego_tags)))

_StartEnd(is_found=True, t_start=191.98, t_end=192.42)


In [11]:
# Cut-in targets:
# 3850: Motorbike from right (29:47, index=1802)
# 4171: Dark passenger car from right (quite far) (32:50, index=1985)
# 5028: Silver passenger car from right (39:18, index=2373)
# 5191: Black van from left (40:53, index=2468)
# 5645: Black passenger car from left (42:26, index=2562)
# 6008: Gray VW from left (44:26, index=2681)
e = ego_ngram.ngram
for i, t in enumerate(target_ngrams.ngrams):
    search = find_sequence((t, e), (target_tags, ego_tags))
    if search.is_found:
        print(i, search)

554 _StartEnd(is_found=True, t_start=191.98, t_end=192.42)
557 _StartEnd(is_found=True, t_start=190.82, t_end=191.05)
566 _StartEnd(is_found=True, t_start=193.62, t_end=193.8)
575 _StartEnd(is_found=True, t_start=193.41, t_end=193.52)
3850 _StartEnd(is_found=True, t_start=1798.58, t_end=1802.5)
4012 _StartEnd(is_found=True, t_start=1904.91, t_end=1905.34)
4015 _StartEnd(is_found=True, t_start=1904.81, t_end=1905.05)
4171 _StartEnd(is_found=True, t_start=1980.72, t_end=1988.34)
4708 _StartEnd(is_found=True, t_start=2212.68, t_end=2214.96)
5028 _StartEnd(is_found=True, t_start=2369.42, t_end=2398.85)
5040 _StartEnd(is_found=True, t_start=2369.29, t_end=2370.29)
5057 _StartEnd(is_found=True, t_start=2433.01, t_end=2447.91)
5191 _StartEnd(is_found=True, t_start=2464.46, t_end=2480.84)
5341 _StartEnd(is_found=True, t_start=2483.0, t_end=2484.86)
5399 _StartEnd(is_found=True, t_start=2517.22, t_end=2518.19)
5645 _StartEnd(is_found=True, t_start=2557.2, t_end=2565.19)
5766 _StartEnd(is_found=

In [18]:
AD.targets[554].loc[191.98:192.42]

,age,ax,dx,dy,id,lateral_activity,line_center,line_left,line_left_diff,line_left_down,...,prob,speed_dec,speed_dec_start,speed_inc,speed_inc_start,theta,vx,longitudinal_state,lateral_state,lead_vehicle
time,,,,,,,,,,,,,,,,,,,,,
191.98,3.899112e+09,-0.648077,90.436883,6.116486,709.0,ri,-2.239528,-0.749742,0.025581,-0.145760,...,1.0,-0.063159,0.000000,0.032008,0.000000,-0.023006,29.347847,f,l,n
191.99,3.927190e+09,-0.648077,90.741311,6.179297,709.0,ri,-2.211238,-0.724161,0.115057,-0.120179,...,1.0,-0.063159,0.000000,0.032008,0.000000,-0.022108,29.329650,f,l,n
192.00,3.927190e+09,-0.648077,90.741311,6.179297,709.0,ri,-2.051024,-0.609104,-0.004156,-0.005122,...,1.0,-0.064554,0.000000,0.030613,0.000000,-0.022108,29.329650,f,l,n
192.01,3.922622e+09,-0.648077,90.691695,6.169169,709.0,ri,-2.055692,-0.613260,0.000000,-0.009278,...,1.0,-0.064554,0.000000,0.030613,0.000000,-0.022254,29.332611,f,l,n
192.02,3.922622e+09,-0.648077,90.691695,6.169169,709.0,ri,-2.055692,-0.613260,0.058556,-0.009278,...,1.0,-0.066379,0.000000,0.028788,0.000000,-0.022254,29.332611,f,l,n
192.03,3.989253e+09,-0.648077,91.412131,6.320357,709.0,ri,-1.989652,-0.554704,0.000000,0.000000,...,1.0,-0.066379,0.000000,0.028788,0.000000,-0.020122,29.289429,f,l,n
192.04,3.989253e+09,-0.648077,91.412131,6.320357,709.0,ri,-1.989652,-0.554704,0.004053,0.000000,...,1.0,-0.083921,0.000000,0.011245,0.000000,-0.020122,29.289429,f,l,n
192.05,3.993851e+09,-0.648077,91.461562,6.330880,709.0,ri,-1.985082,-0.550652,0.000000,0.000000,...,1.0,-0.083921,0.000000,0.011245,0.011245,-0.019975,29.286448,f,l,n
192.06,3.993851e+09,-0.648077,91.461562,6.330880,709.0,ri,-1.985082,-0.550652,-0.085757,0.000000,...,1.0,-0.083921,0.000000,0.011245,0.011245,-0.019975,29.286448,f,l,n


In [ ]:
i = 4012
target_ngrams.ngrams[i]

In [ ]:
e.loc[target_ngrams.ngrams[i].index[0]-10:target_ngrams.ngrams[i].index[-1]+10]

In [ ]:
plt.plot(AD.targets[i]["line_left"])
plt.plot(AD.targets[i]["line_right"])

In [ ]:
plt.plot(-AD.targets[i]["dy"], AD.targets[i]["dx"], '.')

In [ ]:
approx_index(31, 30, AD.data)

In [ ]:
target_ngrams.ngrams[5191]